In [2]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier,RidgeClassifier,PassiveAggressiveClassifier
import pickle
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from datetime import datetime
import matplotlib.pyplot as plt
#from pandas_ml import ConfusionMatrix 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix 
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [75]:
input_df = pd.read_csv('incident_1month_resolved.csv')
input_df = input_df[(input_df["Source"] == "Email")]
#input_df = input_df[(input_df['Symptom'].str.contains("Brian Shepard"))]
input_df = input_df.reset_index()
print(input_df.shape)
# input_df = input_df.sample(1000)
# input_df = input_df.reset_index()

input_df['Symptom'] = input_df['Symptom'].replace(regex='<[^>]+>',value='')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\n\\s*)+',value=' \n ')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\r\\s*)+',value=' \r ')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\t\\s*)+',value=' ')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\r\\n\\s*)+',value=' \r\n ')
input_df['Symptom'] = input_df['Symptom'].replace(regex=["\\sFrom:.*\\s",
                        "\\sTo:.*\\s","\\sDate:.*\\s","\\sSubject:.*\\s","\\sSent:.*\\s",
                        "\\sCc:.*\\s","\\sBcc:\\s"], value='')
input_df['Symptom'] = input_df['Symptom'].replace(regex=[                                                         
                        "(?s)\\sThis message is intended .* destroy all copies\\.?\\s",
                        "(?s)\\sDisclaimer .* only if necessary\\.?\\s",
                        "(?s)\\sConfidentiality Note:.*original message\\.?\\s"], value='')
input_df['Symptom'] = input_df['Symptom'].replace(regex='\\(AutoClosed\\)', value = '')

# input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\-\\s*)+',value='-')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\+',value=' PLUS ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\|',value=' ORR ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='[\\s]+',value=' ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\-',value=' HYP ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\(',value=' OB ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\)',value=' CB ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\-',value=' HYP ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\@',value=' ATTHE ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\$',value=' DOLLAR ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\.',value=' . ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex=':',value=' : ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\[',value=' [ ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\]',value=' ] ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\<',value=' < ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\>',value=' > ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\,',value=' , ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\'',value=' ' ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\=',value=' = ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\/',value=' / ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\?',value=' ? ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex="'",value="\\'")
# input_df['Symptom'] = input_df['Symptom'].replace(regex='"',value='\\"')

input_df['Symptom'] = input_df['Symptom'].str.lower()
input_df['Symptom'] = input_df['Symptom'].replace(regex='[\\s]+',value=' ')


#input_df.to_csv("testing.csv")
#print(input_df['Symptom'][4486])

(11304, 43)


In [76]:
for i in range(20):
    print(input_df["Symptom"][i])
    print("\n-------\n")

best regards, ralph schoell manager, infrastructure systems vmware certified professional on vsphere 4/5 netapp certified data management administrator dsm 7 certified engineer need it assistance? please call the help line @ (+1.719.268.)4357 or submit an incident 

-------

whenever i send an email to brendan.kelley@heatsoftware.com from outlook, i get the following error. i am able to send him emails from my phone without any issues. i deleted his saved email address from outlook but i still have issues. i need to get this fixed asap please. thanks, brian brian shepard information technology – enterprise apps heat software 425.210.2637 direct brian.shepard@heatsoftware.com | www.heatsoftware.com _____________________________________________ the email address you entered couldn't be found. please check the recipient's email address and try to resend the message. if the problem continues, please contact your helpdesk. diagnostic information for administrators: generating server: cs-mbx

In [ ]:
# result = pd.read_csv('success5.csv')
# for i in range(result.shape[0]):
#     if result["Symptom"][i] != result["New_Symptom"][i]:
#         print(result["Symptom"][i])
#         print("\n######\n")
#         print(result["New_Symptom"][i])
#         print("\n--------\n")
#     if i == 30:
#         break

In [ ]:
# input_df["Symptom"].str.contains("from the available options . macintosh users : 1 . click on the apple menu and select system preferences 2 . click on users changed your password on the domain . password restrictions : network passwords are valid for a period of 90 days password cannot be any of the 5 previously used passwords passwords must contain 3 of the 4 types of characters uppercase ob example : a cb lowercase ob example : a cb number ob example : 1 cb symbol ob example : $ , @ , etc . cb if you have any questions or need further assistance , please contact the it service desk by logging an incident through it . heatsoftware . com or call the help line at ext . 4357 . thank you , it service desk heat software phone : plus 1 . 719 . 268 . help ob 4357 cb need it assistance ? please call the").sum()

In [77]:
import re
import re
from datetime import datetime
import itertools
def countTotal(ser,vecs):
    print(len(vecs))
    total = 0;
    for vec in vecs:
        a = ser.str.contains(vec).sum()
        print(a)
        #print(ser)
        #print(ser.str.contains("original"))
        total += a
    print("Total occurances - %d" %total)
    
def parse_text(text):
    for pattern, replacement in new_dict_comp.items():
        text = pattern.sub(replacement, text)
    return text

i = 50;
startTime1 = datetime.now()
input_df["New_Symptom"] = input_df["Symptom"];
for i in range(300,9,-1):
    for j in range(5,4,-1):
        startTime = datetime.now()
        startTime2 = datetime.now()
        
        print(i , j)
        try:
            pattern = "[\\S]+";
            tf_vectorizer = CountVectorizer( min_df=j,
                                            ngram_range = (i,i),
                                            analyzer='word',
                                            tokenizer=lambda x: x.split(' ')
                                            )
            tf_vectorizer.fit(input_df["New_Symptom"])
            #print(tf_vectorizer.get_feature_names())
            print("total vector size - %d" %(len(tf_vectorizer.get_feature_names())))
            
            startTime3 = datetime.now()
            
            a =  zip(tf_vectorizer.get_feature_names(), itertools.repeat(""))
            new_dict = {re.escape(k): v for k, v in a}
            new_dict_comp = {re.compile(k): v for k, v in new_dict.items()}
#             countTotal(input_df["New_Symptom"],tf_vectorizer.get_feature_names())
            input_df["New_Symptom"] = input_df["New_Symptom"].apply(parse_text)
            
#             for vec in tf_vectorizer.get_feature_names():
#                 #print(input_df["New_Symptom"].str.contains(vec).sum())
#                 input_df["New_Symptom"] = input_df["New_Symptom"].str.replace(re.escape(vec),"")  

            print("time to replace - ")
            print(datetime.now() - startTime3)
            #countTotal(input_df["New_Symptom"],tf_vectorizer.get_feature_names())
#             pattern = "[\\S]+";
#             tf_vectorizer = CountVectorizer( min_df=j,
#                                             ngram_range = (i,i),
#                                             analyzer='word',
#                                             tokenizer=lambda x: x.split(' ')
#                                             )
#             tf_vectorizer.fit(input_df["New_Symptom"])
#             print("total vector size after replacement - %d" %(len(tf_vectorizer.get_feature_names())))
#             #print(tf_vectorizer.get_feature_names()[0])
#             #input_df["New_Symptom"] = temp
#             print("time to vectorize 2 times- ") 
#             print(datetime.now() - startTime2)
        except ValueError:
            print(ValueError)
        except Exception as e:
            print(e)
        print(datetime.now() - startTime)
print("Total Time: - ")
print(datetime.now() - startTime1)


300 5
total vector size - 180
time to replace - 
0:00:01.799177
0:00:03.606813
299 5
<class 'ValueError'>
0:00:01.820164
298 5
<class 'ValueError'>
0:00:01.753918
297 5
<class 'ValueError'>
0:00:01.695506
296 5
<class 'ValueError'>
0:00:01.746634
295 5
<class 'ValueError'>
0:00:01.758338
294 5
<class 'ValueError'>
0:00:01.758719
293 5
<class 'ValueError'>
0:00:01.741884
292 5
<class 'ValueError'>
0:00:01.729174
291 5
<class 'ValueError'>
0:00:01.749478
290 5
<class 'ValueError'>
0:00:01.752879
289 5
<class 'ValueError'>
0:00:01.807237
288 5
<class 'ValueError'>
0:00:01.780333
287 5
<class 'ValueError'>
0:00:01.764015
286 5
<class 'ValueError'>
0:00:01.780691
285 5
<class 'ValueError'>
0:00:01.740551
284 5
<class 'ValueError'>
0:00:01.754822
283 5
<class 'ValueError'>
0:00:01.868489
282 5
<class 'ValueError'>
0:00:01.804585
281 5
<class 'ValueError'>
0:00:01.792893
280 5
<class 'ValueError'>
0:00:01.873627
279 5
<class 'ValueError'>
0:00:01.833331
278 5
<class 'ValueError'>
0:00:01.7730

<class 'ValueError'>
0:00:02.843977
112 5
<class 'ValueError'>
0:00:02.665094
111 5
<class 'ValueError'>
0:00:02.672068
110 5
<class 'ValueError'>
0:00:02.739551
109 5
<class 'ValueError'>
0:00:02.748507
108 5
total vector size - 1
time to replace - 
0:00:00.022576
0:00:02.606829
107 5
<class 'ValueError'>
0:00:02.906034
106 5
total vector size - 1
time to replace - 
0:00:00.023504
0:00:02.610669
105 5
total vector size - 1
time to replace - 
0:00:00.025296
0:00:02.852577
104 5
total vector size - 2
time to replace - 
0:00:00.033908
0:00:02.856468
103 5
<class 'ValueError'>
0:00:02.943560
102 5
<class 'ValueError'>
0:00:02.798156
101 5
<class 'ValueError'>
0:00:02.905037
100 5
total vector size - 2
time to replace - 
0:00:00.037092
0:00:02.740087
99 5
total vector size - 1
time to replace - 
0:00:00.025083
0:00:02.949206
98 5
total vector size - 2
time to replace - 
0:00:00.033640
0:00:02.949715
97 5
<class 'ValueError'>
0:00:03.262976
96 5
<class 'ValueError'>
0:00:02.983323
95 5
tota

In [81]:
input_df.to_csv("results2.csv")

In [22]:
input_df = pd.read_csv('results2.csv')


In [14]:
type(input_df["New_Symptom"][1])

str

In [20]:
sum(input_df["Symptom"] != input_df["New_Symptom"])

7124

In [80]:
j = 0
for i in range(30,input_df.shape[0]):
    if input_df["Symptom"][i] != input_df["New_Symptom"][i]:
        j += 1
        print(input_df["Symptom"][i])
        print("\n######\n")
        print(input_df["New_Symptom"][i])
        print("\n--------\n")
    if j == 20:
        break

hi, i seem to have misplaced my door fob and i would need a new one. i left for holiday and left the fob at home, but today i couldn’t find it. i will have another look, but in the meantime, could i get a new one? thanks, michal michal krzywonos technical support engineer – emea t +1 719 278 7170 f +353 9176 6722 w www.ivanti.com note: my new email address is: michal.krzywonos@ivanti.com 

######

hi, i seem to have misplaced my door fob and i would need a new one. i left for holiday and left the fob at home, but today i couldn’t find it. i will have another look, but in the meantime, could i get a new one? thanks, michal 

--------

kind regards, link lee information technologies | asia pac zhenhan.li@ivanti.com tel: 8610-85153668 ext 226 e-fax:8610-85187107 fax:8610-85183135 

######



--------

hi! is my outlook not configured properly? i receive this cancellation of one recurring meeting but it does not tell me which occurrence is cancelled : how do i know which meeting is cancell

In [42]:
test = input_df[(input_df['Symptom'].str.contains("benjamin elzingre"))]
print(test.shape)

(105, 44)


In [46]:

for i in test["Symptom"]:
    print(i)
    print("\n--------\n")
   

good morning, can you please contact me as soon as possible regarding this issue? attached is the screen michal gets when he boots his pc he is just back from holiday and nothing was done to the machine as far as we can tell thank you for your help benjamin elzingre technical support manager – emea ivanti benjamin.elzingre@ivanti.com | www.heatsoftware.com | www.ivanti.com

--------

hello, could you please add bettina to the “_gss endpoint security intl” dl please? cheers, benjamin elzingre technical support manager – emea ivanti benjamin.elzingre@ivanti.com | www.heatsoftware.com | www.ivanti.com

--------

hello, it seems something is changing my password regularly because my account got locked again can you please check it and give me access again? cheers, benjamin elzingre technical support manager – emea ivanti benjamin.elzingre@ivanti.com | www.heatsoftware.com | www.ivanti.com

--------

hello, during the upgrade of my laptop i seem to have lost the agent and the console instal

In [23]:
def remove_and_dedup_punctuation_numbers(text): 
    result = text.replace('[\w\d\.-]+@[\w\d\.-]+',' ').replace(
    '\d+', ' ').replace('_+', ' ').replace('+', ' ').replace(
    '.', ' ').replace('^https?:\/\/.*[\r\n]*', '').replace('[^\w]',' ').lower()
    result = re.sub (r'([^a-zA-Z\s.?!])', '', result)
    result = re.sub(r"([" + re.escape(string.punctuation) + r"])\1+", r"\1", result)
    result = re.sub( '\s+', ' ', result ).strip()
    result = re.sub(r'<[^>]+>', '', result)
    return result

#input_df = input_df.dropna()
input_df["New_Symptom"] = input_df["New_Symptom"].apply(remove_and_dedup_punctuation_numbers)
input_df.shape

AttributeError: 'float' object has no attribute 'replace'

In [1]:
no_features = 30
tf_vectorizer1 = CountVectorizer(max_df=0.90, min_df=10, ngram_range=(1,2), max_features=no_features, stop_words='english')
tf = tf_vectorizer1.fit_transform(input_df['New_Symptom'].values.astype('U'))
tf_feature_names = tf_vectorizer1.get_feature_names()
print(len(tf_feature_names))
weights = np.asarray(tf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tf_feature_names, 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(30)

NameError: name 'CountVectorizer' is not defined

In [68]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (",".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [98]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
no_topics = 20
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
access,login,email,frs,netsuite,new,fs,cs,fw,working
Topic 1:
notification,service,email,request,need,phone,vpn,password,server,date
Topic 2:
frs,alert,server,2015,fw,fs,cs,start,10,email
Topic 3:
new,password,working,access,need,request,cs,heat,service,phone
Topic 4:
2015,date,10,start,notification,new,server,fw,frs,alert
Topic 5:
fw,service,request,netsuite,issue,account,frs,alert,working,unable
Topic 6:
login,fs,frs,alert,unable,outlook,issue,password,com,webex
Topic 7:
unable,heat,access,account,server,working,2015,service,10,outlook
Topic 8:
need,account,cs,webex,netsuite,new,access,heat,server,frs
Topic 9:
com,server,fs,access,10,cs,date,frs,2015,alert
Topic 10:
issue,notification,email,account,phone,server,working,10,access,fs
Topic 11:
account,com,working,need,heat,webex,notification,login,access,service
Topic 12:
cs,outlook,10,server,access,alert,date,2015,frs,service
Topic 13:
account,password,netsuite,request,com,webex,fw,notification,vpn,2015
Topic 14:
webex,need,l

In [99]:
from sklearn.cluster import KMeans
k = 50
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(tf)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tf_vectorizer1.get_feature_names()
for i in range(k):
    print("Cluster %d:" % i, end='\t')
    for ind in order_centroids[i, :10]:
        print('%s' % terms[ind], end=',')
    print()

Top terms per cluster:
Cluster 0:	server,fw,10,com,heat,alert,cs,fs,webex,issue,
Cluster 1:	access,need,com,request,heat,email,issue,fs,cs,new,
Cluster 2:	notification,2015,frs,10,fs,alert,start,date,issue,access,
Cluster 3:	cs,frs,2015,server,access,unable,fs,working,issue,account,
Cluster 4:	request,service,new,working,heat,2015,access,account,alert,com,
Cluster 5:	date,notification,start,2015,fw,10,issue,access,account,alert,
Cluster 6:	fs,frs,alert,login,webex,2015,access,account,com,cs,
Cluster 7:	outlook,password,webex,issue,working,new,login,request,email,need,
Cluster 8:	2015,server,10,frs,fs,issue,access,account,alert,com,
Cluster 9:	com,login,fw,email,unable,working,webex,account,password,access,
Cluster 10:	login,unable,netsuite,issue,vpn,fw,password,heat,working,webex,
Cluster 11:	new,request,webex,email,phone,account,heat,server,working,password,
Cluster 12:	fw,heat,notification,account,request,phone,issue,vpn,alert,service,
Cluster 13:	issue,webex,phone,email,vpn,account,